## Table of contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref2">Part 1: Quadratic Cost Function </a></li>
        <ul>
         <li><a href="#ref3">N1 = 2, 4, 6, 8 ,10 </a></li>
         <li><a href="#ref3">Observations for N1 = 2, 4, 6, 8 ,10 </a></li>
         <li><a href="#ref4">N1 = 1</a></li>
         <li><a href="#ref3">Observations for N1 = 1 </a></li>
        </ul>
        <li><a href="#ref2">Part 2: Bipolar Cost Function </a></li>
        <ul>
         <li><a href="#ref3">N1 = 2, 4, 6, 8 ,10 </a></li>
         <li><a href="#ref3">Observations for N1 = 2, 4, 6, 8 ,10 </a></li>
         <li><a href="#ref4">N1 = 1</a></li>
         <li><a href="#ref3">Observations for N1 = 1 </a></li>
        </ul>
    </ol>
</div>
<br>

1. Termination Condition
2. Proper Epoch Finding
3. 4 hyper-parameters Search

4 hyper-parameters of the problem: N1, α, ζ, and x0. Perform calculations using the following choices of parameter values: α = 0.1, 0.2, 0.3, ζ = 0.5, 1.0, 1.5, and x0 =0.5,1.0,1.5.

Separately for N1 = 2, 4, 6, 8, 10, run your program 100 times, each time using randomly created weights and biases. Use the same training rate and T as before. For a given N1, report the number of times there is no convergence. For the remaining cases where you have convergent results, report the number of epochs needed, the average and median number of epochs. as well as the minimum and maximum number of epochs. Comment on your findings.

# Part 1

## Quadratic Cost Function for N1= 2, 4, 6, 8, 10

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import math
class MLP:
	def __init__(self, w1, b1, w2, b2, lr,x0):
		self.l1 = FCLayer(w1, b1, lr)
		self.a1 = Sigmoid(x0)
		self.l2 = FCLayer(w2, b2, lr)
		self.a2 = Sigmoid(x0)

	def MSE(self, prediction, target):
		return 0.5*np.square(target - prediction).sum()

	def MSEGrad(self, prediction, target):
		return - (target - prediction)

	def shuffle(self, X, y):
		idxs = np.arange(y.size)
		np.random.shuffle(idxs)
		return X[idxs], y[idxs]

	def train(self, X, y, epochs,T):
		flag=0
		for i in range(epochs):
			loss=0
			if flag==1:
				print("--------------")
				break
			for s in range(y.size):
				m = s % y.size
				xi = np.expand_dims(X[m], axis=0)
				yi = np.expand_dims(y[m], axis=0)
				pred = self.l1.forward(xi)
				pred = self.a1.forward(pred)
				pred = self.l2.forward(pred)
				pred = self.a2.forward(pred)
				loss = loss+self.MSE(pred, yi) 
				grad = self.MSEGrad(pred, yi)
				grad = self.a2.backward(grad)
				grad= self.l2.backward(grad)
				grad = self.a1.backward(grad)
				grad= self.l1.backward(grad)
			if loss<T:
				flag=1
				return i+1
				break
		return i+1
    
	def predict(self, X):
		pred = self.l1.forward(X)
		pred = self.a1.forward(pred)
		pred = self.l2.forward(pred)
		pred = self.a2.forward(pred)
		pred = np.round(pred)
		return np.ravel(pred)

class FCLayer:

	def __init__(self, w, b, lr):
		self.lr = lr
		self.w = w	#Each column represents all the weights going into an output node
		self.b = b

	def forward(self, input1):
		self.inp=input1
		return self.inp@self.w +self.b

	def backward(self, gradients):
		newW=np.transpose(self.inp)@gradients
		newX=gradients@np.transpose(self.w)
		self.w=self.w-self.lr*newW
		self.b=self.b -self.lr*gradients
		return newX

class Sigmoid:

	def __init__(self,x0):
		self.x0=x0

	def forward(self, x):
		#Write forward pass here
		# ------- To Remove Numpy Overflow Warning ---------
		max1 = np.log(np.finfo('float64').max)
		min1 = -max1
		x[x>max1] = max1
		x[x<min1] = min1
		# ------------------------------------------------
		self.sig=(1-np.exp(-1*(x/self.x0)))/(1 + np.exp(-1*(x/self.x0)))
		return self.sig

	def backward(self, gradients):
		#Write backward pass here
		self.devsig=gradients*(1-self.sig)*(1+self.sig)*(1/(2*self.x0))
		return self.devsig

In [2]:
from collections import defaultdict
val = defaultdict(list)
#------------------------------ Training Data -----------------------------------
X=np.array([[1,1],[1,-1],[-1,1],[-1,-1]])
y=np.array([-1,1,1,-1])
#------------------------------ Hyper Parameter -----------------------------------
epochs=2000
T=0.05
lr1=[0.1,0.2,0.3]
weightspace=[0.5,1.0,1.5]
par_x0=[0.5,1.0,1.5]
#----------------------------- Neural Net Structure ---------------------------
N0=2
N1=[1,2,4,6,8,10]
N2=1
for net in N1:
    #--------------------------------Running 100 times for each N1 ---------------------
    for j in range(100):
        #print("\n----------------------------------------------\n Running",j)
        for weight in weightspace:
            w1 = np.random.uniform(-1*weight, weight, size=(N0,net))
            w2 = np.random.uniform(-1*weight, weight, size=(net,N2))
            b1 = np.random.uniform(-1*weight, weight, size=(1,net))
            b2 = np.random.uniform(-1*weight, weight, size=(N2,N2))
            for alpha in lr1:
                for x0 in par_x0:
                    #--------------------------- Initializing the class ---------------------------
                    mlp = MLP(w1, b1, w2, b2, alpha,x0)
                    #--------------------------------- Training ----------------------------
                    ep=mlp.train(X, y, epochs,T)
                    #-------------------------------- Prediction ----------------------------------------
                    if (ep == epochs):
                        converge = 0
                    else:
                        converge = 1                    
                    #solutions = mlp.predict(X)
                    #acc=np.mean(solutions==y)
                    val[(net,weight,alpha,x0,converge)].append(ep)
                    #print ("Accuracy is {} Learning Rate is {} epoch {}".format(acc,alpha,ep))


In [3]:
lis=[]
for key in val.keys():
    temp=np.array(val[key])
    lis.append([key,temp.mean(),np.median(temp),np.min(temp),np.max(temp),temp.shape[0]])
df1=pd.DataFrame(lis,columns=['key','mean_epoch','median_epoch','min_epoch','max_epoch','Result'])
['N1', 'weightspace','alpha','x0','converge']
df1.loc[:,'N1']=df1.key.map(lambda x:x[0])
df1.loc[:,'weightspace']=df1.key.map(lambda x:x[1])
df1.loc[:,'alpha']=df1.key.map(lambda x:x[2])
df1.loc[:,'x0']=df1.key.map(lambda x:x[3])
df1.loc[:,'converge']=df1.key.map(lambda x:x[4])
df1['converge']=df1['converge'].map(lambda x : "Converged for : " if (x==1) else "Did not converged : ")
df1['Result'] = df1['converge'] + " "+df1['Result'].map(str)+" times "
cols = list(df1.columns)
df1 = df1[cols[-5:]+cols[:-5]]
df1=df1.drop(['key','converge'],axis=1)
df1=df1.sort_values(by=['N1','weightspace','alpha','x0','Result'])
#df1.to_csv("quadratic.csv")
# df1.head()

### N1 = 2 quadratic cost function

In [4]:
df1[df1.N1==2]

,N1,weightspace,alpha,x0,mean_epoch,median_epoch,min_epoch,max_epoch,Result
27,2,0.5,0.1,0.5,125.809524,119.0,82,238,Converged for : 63 times
61,2,0.5,0.1,0.5,2000.000000,2000.0,2000,2000,Did not converged : 37 times
28,2,0.5,0.1,1.0,787.388889,680.5,456,1882,Converged for : 54 times
62,2,0.5,0.1,1.0,2000.000000,2000.0,2000,2000,Did not converged : 46 times
54,2,0.5,0.1,1.5,1736.117647,1768.0,1401,1976,Converged for : 17 times
29,2,0.5,0.1,1.5,2000.000000,2000.0,2000,2000,Did not converged : 83 times
30,2,0.5,0.2,0.5,63.171429,61.0,42,94,Converged for : 35 times
63,2,0.5,0.2,0.5,2000.000000,2000.0,2000,2000,Did not converged : 65 times
65,2,0.5,0.2,1.0,353.944444,346.0,249,503,Converged for : 18 times
31,2,0.5,0.2,1.0,2000.000000,2000.0,2000,2000,Did not converged : 82 times


### N1 = 4 quadratic cost function

In [5]:
df1[df1.N1==4]

,N1,weightspace,alpha,x0,mean_epoch,median_epoch,min_epoch,max_epoch,Result
81,4,0.5,0.1,0.5,97.055556,94.0,58,166,Converged for : 90 times
123,4,0.5,0.1,0.5,2000.000000,2000.0,2000,2000,Did not converged : 10 times
82,4,0.5,0.1,1.0,672.627907,598.5,416,1793,Converged for : 86 times
124,4,0.5,0.1,1.0,2000.000000,2000.0,2000,2000,Did not converged : 14 times
117,4,0.5,0.1,1.5,1685.540541,1712.0,1255,1998,Converged for : 37 times
83,4,0.5,0.1,1.5,2000.000000,2000.0,2000,2000,Did not converged : 63 times
118,4,0.5,0.2,0.5,54.500000,53.5,36,95,Converged for : 60 times
84,4,0.5,0.2,0.5,2000.000000,2000.0,2000,2000,Did not converged : 40 times
119,4,0.5,0.2,1.0,356.861111,351.0,223,637,Converged for : 36 times
85,4,0.5,0.2,1.0,2000.000000,2000.0,2000,2000,Did not converged : 64 times


### N1 = 6 quadratic cost function

In [6]:
df1[df1.N1==6]

,N1,weightspace,alpha,x0,mean_epoch,median_epoch,min_epoch,max_epoch,Result
128,6,0.5,0.1,0.5,80.907216,79.0,55,200,Converged for : 97 times
174,6,0.5,0.1,0.5,2000.000000,2000.0,2000,2000,Did not converged : 3 times
129,6,0.5,0.1,1.0,564.106383,516.5,368,1536,Converged for : 94 times
170,6,0.5,0.1,1.0,2000.000000,2000.0,2000,2000,Did not converged : 6 times
130,6,0.5,0.1,1.5,1570.220339,1533.0,1125,1998,Converged for : 59 times
157,6,0.5,0.1,1.5,2000.000000,2000.0,2000,2000,Did not converged : 41 times
131,6,0.5,0.2,0.5,59.595238,51.0,33,432,Converged for : 84 times
158,6,0.5,0.2,0.5,2000.000000,2000.0,2000,2000,Did not converged : 16 times
132,6,0.5,0.2,1.0,309.800000,285.0,201,574,Converged for : 55 times
159,6,0.5,0.2,1.0,2000.000000,2000.0,2000,2000,Did not converged : 45 times


### N1 = 8 quadratic cost function

In [7]:
df1[df1.N1==8]

,N1,weightspace,alpha,x0,mean_epoch,median_epoch,min_epoch,max_epoch,Result
176,8,0.5,0.1,0.5,74.591837,69.0,47,309,Converged for : 98 times
220,8,0.5,0.1,0.5,2000.000000,2000.0,2000,2000,Did not converged : 2 times
177,8,0.5,0.1,1.0,515.135417,503.0,336,1001,Converged for : 96 times
211,8,0.5,0.1,1.0,2000.000000,2000.0,2000,2000,Did not converged : 4 times
178,8,0.5,0.1,1.5,1537.969697,1497.0,1010,1988,Converged for : 66 times
208,8,0.5,0.1,1.5,2000.000000,2000.0,2000,2000,Did not converged : 34 times
179,8,0.5,0.2,0.5,94.010870,52.0,26,1409,Converged for : 92 times
216,8,0.5,0.2,0.5,2000.000000,2000.0,2000,2000,Did not converged : 8 times
180,8,0.5,0.2,1.0,305.184615,278.0,182,515,Converged for : 65 times
209,8,0.5,0.2,1.0,2000.000000,2000.0,2000,2000,Did not converged : 35 times


### N1 = 10 quadratic cost function

In [8]:
df1[df1.N1==10] 

,N1,weightspace,alpha,x0,mean_epoch,median_epoch,min_epoch,max_epoch,Result
221,10,0.5,0.1,0.5,83.758242,65.0,45,516,Converged for : 91 times
258,10,0.5,0.1,0.5,2000.000000,2000.0,2000,2000,Did not converged : 9 times
222,10,0.5,0.1,1.0,486.666667,450.0,316,1825,Converged for : 99 times
262,10,0.5,0.1,1.0,2000.000000,2000.0,2000,2000,Did not converged : 1 times
223,10,0.5,0.1,1.5,1464.555556,1451.0,1081,1991,Converged for : 81 times
253,10,0.5,0.1,1.5,2000.000000,2000.0,2000,2000,Did not converged : 19 times
224,10,0.5,0.2,0.5,169.658537,49.0,23,1653,Converged for : 82 times
254,10,0.5,0.2,0.5,2000.000000,2000.0,2000,2000,Did not converged : 18 times
225,10,0.5,0.2,1.0,301.839506,274.0,178,868,Converged for : 81 times
255,10,0.5,0.2,1.0,2000.000000,2000.0,2000,2000,Did not converged : 19 times


## Observations for N1= 2,4,6,8,10 with Quadratic Cost Function

1. ***It is observed that the random nature of weights has a lot of influence in convergence or non-convergence. For some weights, the neural net never converges.***


2. ***For - ζ and ζ in weightspace. It is observed that as ζ increase initial weight and bias range will result in fewer non-converge. It shows that a little larger weights and bias will encourage the better performance of our backpropagation model.***


3. ***For all test cases, As x0 increases, in most cases, the min, max, mean and median of epochs all increases which means that the model generally takes more iterations to converge.***


4. ***In extreme cases, when x0 = 1.5 and learning rate = 0.1, it is very hard for the model to converge. As x0 is a parameter of the transfer function, when x0 get larger, the slope of both transfer function and its derivative become less steep. In this way, it will affect the gradient of the backpropagation process and result in more iterations to converge.***


5. ***For all test cases, a larger learning rate will result in faster convergence and fewer iterations. At the same time, a larger learning rate will result in more non-convergence. This is reasonable because a large learning rate could substantially accelerate the learning process and if the step is too large(in other word, learning rate too large), the network will fail to converge.***
6. ***The larger the N1, the fewer epochs are required for convergence***

## ----------------------------------------------------------------------

## N1=1 for Quadratic Cost Function

### 4.1 Consider next the case where N1 = 1. What do you find? Can you explain it?

In [9]:
df1[df1.N1==1]

,N1,weightspace,alpha,x0,mean_epoch,median_epoch,min_epoch,max_epoch,Result
0,1,0.5,0.1,0.5,2000.0,2000.0,2000,2000,Did not converged : 100 times
1,1,0.5,0.1,1.0,2000.0,2000.0,2000,2000,Did not converged : 100 times
2,1,0.5,0.1,1.5,2000.0,2000.0,2000,2000,Did not converged : 100 times
3,1,0.5,0.2,0.5,2000.0,2000.0,2000,2000,Did not converged : 100 times
4,1,0.5,0.2,1.0,2000.0,2000.0,2000,2000,Did not converged : 100 times
5,1,0.5,0.2,1.5,2000.0,2000.0,2000,2000,Did not converged : 100 times
6,1,0.5,0.3,0.5,2000.0,2000.0,2000,2000,Did not converged : 100 times
7,1,0.5,0.3,1.0,2000.0,2000.0,2000,2000,Did not converged : 100 times
8,1,0.5,0.3,1.5,2000.0,2000.0,2000,2000,Did not converged : 100 times
9,1,1.0,0.1,0.5,2000.0,2000.0,2000,2000,Did not converged : 100 times


### Observation:

- ***When N1 = 1, the experiment never converge. Because XOR is not linearly separable***

## -------------------------------------------------------------------------------

# Part 2 - Bipolar Cost Function

### 2.1 Repeat the above computation by replacing the use of the quadratic cost function with the bipolar cross-entropy cost function (still using the same termination conditions). I have updated the lecture notes on backpropagation by including the bipolar cross entropy function (slide 17). Report your findings. What comments can you make?

In [10]:
import numpy as np
class MLP:
	def __init__(self, w1, b1, w2, b2, lr,x0):
		self.l1 = FCLayer(w1, b1, lr)
		self.a1 = Sigmoid(x0)
		self.l2 = FCLayer(w2, b2, lr)
		self.a2 = Sigmoid(x0)

	def MSE(self, prediction, target):
		return -2*np.log2(0.5*abs(target+prediction))

	def MSEGrad(self, prediction, target):
		return - 2/((math.log(2)*(target + prediction)))

	def shuffle(self, X, y):
		idxs = np.arange(y.size)
		np.random.shuffle(idxs)
		return X[idxs], y[idxs]

	def train(self, X, y, epochs,T):
		flag=0
		for i in range(epochs):
			loss=0
			if flag==1:
				print("--------------")
				break
			for s in range(y.size):
				m = s % y.size
				xi = np.expand_dims(X[m], axis=0)
				yi = np.expand_dims(y[m], axis=0)

				pred = self.l1.forward(xi)
				pred = self.a1.forward(pred)
				pred = self.l2.forward(pred)
				pred = self.a2.forward(pred)
				loss = loss+self.MSE(pred, yi) 

				grad = self.MSEGrad(pred, yi)
				grad = self.a2.backward(grad)
				grad= self.l2.backward(grad)
				grad = self.a1.backward(grad)
				grad= self.l1.backward(grad)
			#loss=np.square(mlp.predict(X) - y).sum() 
			#print(loss)
			if loss<T:
				#print("loss less than T\n")
				flag=1
				return i+1
				break
		return i+1

	def predict(self, X):
		pred = self.l1.forward(X)
		pred = self.a1.forward(pred)
		pred = self.l2.forward(pred)
		pred = self.a2.forward(pred)
		pred = np.round(pred)
		return np.ravel(pred)

class FCLayer:

	def __init__(self, w, b, lr):
		self.lr = lr
		self.w = w	#Each column represents all the weights going into an output node
		self.b = b

	def forward(self, input1):
		self.inp=input1
		return self.inp@self.w +self.b

	def backward(self, gradients):
		newW=np.transpose(self.inp)@gradients
		newX=gradients@np.transpose(self.w)
		self.w=self.w-self.lr*newW
		self.b=self.b -self.lr*gradients
		return newX

class Sigmoid:

	def __init__(self,x0):
		self.x0=x0

	def forward(self, x):
		#Write forward pass here
		# ------- To Remove Numpy Overflow Warning ---------
		max1 = np.log(np.finfo('float64').max)
		min1 = -max1
		x[x>max1] = max1
		x[x<min1] = min1
		# ------------------------------------------------
		self.sig=(1-np.exp(-1*(x/self.x0)))/(1 + np.exp(-1*(x/self.x0)))
		return self.sig

	def backward(self, gradients):
		#Write backward pass here
		self.devsig=gradients*(1-self.sig)*(1+self.sig)*(1/(2*self.x0))
		return self.devsig

In [11]:
from collections import defaultdict
val = defaultdict(list)
#------------------------------ Training Data -----------------------------------
X=np.array([[1,1],[1,-1],[-1,1],[-1,-1]])
y=np.array([-1,1,1,-1])
#------------------------------ Hyper Parameter -----------------------------------
epochs=2000
T=0.05
lr1=[0.1,0.2,0.3]
weightspace=[0.5,1.0,1.5]
par_x0=[0.5,1.0,1.5]
#----------------------------- Neural Net Structure ---------------------------
N0=2
N1=[1,2,4,6,8,10]
N2=1
for net in N1:
    #--------------------------------Running 100 times for each N1 ---------------------
    for j in range(100):
        #print("\n----------------------------------------------\n Running",j)
        for weight in weightspace:
            w1 = np.random.uniform(-1*weight, weight, size=(N0,net))
            w2 = np.random.uniform(-1*weight, weight, size=(net,N2))
            b1 = np.random.uniform(-1*weight, weight, size=(1,net))
            b2 = np.random.uniform(-1*weight, weight, size=(N2,N2))
            for alpha in lr1:
                for x0 in par_x0:
                    #--------------------------- Initializing the class ---------------------------
                    mlp = MLP(w1, b1, w2, b2, alpha,x0)
                    #--------------------------------- Training ----------------------------
                    ep=mlp.train(X, y, epochs,T)
                    #-------------------------------- Prediction ----------------------------------------
                    if (ep == epochs):
                        converge = 0
                    else:
                        converge = 1 
                    #solutions = mlp.predict(X)
                    #acc=np.mean(solutions==y)
                    val[(net,weight,alpha,x0,converge)].append(ep)
                    #print ("Accuracy is {} Learning Rate is {} epoch {}".format(acc,alpha,ep)) 
lis=[]
for key in val.keys():
    temp=np.array(val[key])
    lis.append([key,temp.mean(),np.median(temp),np.min(temp),np.max(temp),temp.shape[0]])
    #print(key,"|| mean is",temp.mean(),"|| median is",np.median(temp),"|| length is",temp.shape[0])
df=pd.DataFrame(lis,columns=['key','mean_epoch','median_epoch','min_epoch','max_epoch','Result'])
#df.key=df.key.map(lambda x:x[1:-1].split(','))
['N1', 'weightspace','alpha','x0','converge']
df.loc[:,'N1']=df.key.map(lambda x:x[0])
df.loc[:,'weightspace']=df.key.map(lambda x:x[1])
df.loc[:,'alpha']=df.key.map(lambda x:x[2])
df.loc[:,'x0']=df.key.map(lambda x:x[3])
df.loc[:,'converge']=df.key.map(lambda x:x[4])
df['converge']=df['converge'].map(lambda x : "Converged : " if (x==1) else "Did not converged : ")
df['Result'] = df['converge'] + " "+df['Result'].map(str)+" times"
cols = list(df.columns)
df = df[cols[-5:]+cols[:-5]]
df=df.drop(['key','converge'],axis=1)
df=df.sort_values(by=['N1','weightspace','alpha','x0','Result'])
#df.to_csv("Final_bipolar")
# df.head()

### N1 = 2 for Bipolar cross-entropy cost function  

In [22]:
print(df[df.N1==2].shape)
df[df.N1==2]

(50, 9)


,N1,weightspace,alpha,x0,mean_epoch,median_epoch,min_epoch,max_epoch,Result
59,2,0.5,0.1,0.5,181.285714,178.0,172,193,Converged : 7 times
27,2,0.5,0.1,0.5,2000.000000,2000.0,2000,2000,Did not converged : 93 times
60,2,0.5,0.1,1.0,780.400000,759.0,743,839,Converged : 5 times
28,2,0.5,0.1,1.0,2000.000000,2000.0,2000,2000,Did not converged : 95 times
61,2,0.5,0.1,1.5,1881.000000,1882.0,1857,1904,Converged : 3 times
29,2,0.5,0.1,1.5,2000.000000,2000.0,2000,2000,Did not converged : 97 times
30,2,0.5,0.2,0.5,96.964286,95.0,84,121,Converged : 84 times
75,2,0.5,0.2,0.5,2000.000000,2000.0,2000,2000,Did not converged : 16 times
31,2,0.5,0.2,1.0,2000.000000,2000.0,2000,2000,Did not converged : 100 times
32,2,0.5,0.2,1.5,2000.000000,2000.0,2000,2000,Did not converged : 100 times


### N1 = 4 Bipolar cross-entropy cost function   

In [21]:
print(df[df.N1==4].shape)
df[df.N1==4]


(44, 9)


,N1,weightspace,alpha,x0,mean_epoch,median_epoch,min_epoch,max_epoch,Result
110,4,0.5,0.1,0.5,173.727273,172.0,157,194,Converged : 11 times
77,4,0.5,0.1,0.5,2000.000000,2000.0,2000,2000,Did not converged : 89 times
109,4,0.5,0.1,1.0,730.888889,731.0,658,802,Converged : 9 times
78,4,0.5,0.1,1.0,2000.000000,2000.0,2000,2000,Did not converged : 91 times
111,4,0.5,0.1,1.5,1839.833333,1786.0,1756,1988,Converged : 6 times
79,4,0.5,0.1,1.5,2000.000000,2000.0,2000,2000,Did not converged : 94 times
80,4,0.5,0.2,0.5,45.870000,44.0,18,82,Converged : 100 times
81,4,0.5,0.2,1.0,2000.000000,2000.0,2000,2000,Did not converged : 100 times
82,4,0.5,0.2,1.5,2000.000000,2000.0,2000,2000,Did not converged : 100 times
83,4,0.5,0.3,0.5,55.797980,12.0,5,1927,Converged : 99 times


### N1 = 6 Bipolar cross-entropy cost function  

In [14]:
df[df.N1==6]

,N1,weightspace,alpha,x0,mean_epoch,median_epoch,min_epoch,max_epoch,Result
148,6,0.5,0.1,0.5,162.176471,165.0,115,198,Converged : 17 times
121,6,0.5,0.1,0.5,2000.000000,2000.0,2000,2000,Did not converged : 83 times
149,6,0.5,0.1,1.0,684.600000,692.0,608,755,Converged : 15 times
122,6,0.5,0.1,1.0,2000.000000,2000.0,2000,2000,Did not converged : 85 times
153,6,0.5,0.1,1.5,1728.250000,1721.5,1621,1920,Converged : 12 times
123,6,0.5,0.1,1.5,2000.000000,2000.0,2000,2000,Did not converged : 88 times
124,6,0.5,0.2,0.5,16.960000,15.0,6,52,Converged : 100 times
125,6,0.5,0.2,1.0,2000.000000,2000.0,2000,2000,Did not converged : 100 times
126,6,0.5,0.2,1.5,2000.000000,2000.0,2000,2000,Did not converged : 100 times
127,6,0.5,0.3,0.5,8.740000,8.0,5,38,Converged : 100 times


### N1 = 8 Bipolar cross-entropy cost function  

In [15]:
df[df.N1==8]

,N1,weightspace,alpha,x0,mean_epoch,median_epoch,min_epoch,max_epoch,Result
184,8,0.5,0.1,0.5,161.210526,165.0,96,196,Converged : 19 times
157,8,0.5,0.1,0.5,2000.000000,2000.0,2000,2000,Did not converged : 81 times
185,8,0.5,0.1,1.0,657.391304,664.0,409,839,Converged : 23 times
158,8,0.5,0.1,1.0,2000.000000,2000.0,2000,2000,Did not converged : 77 times
186,8,0.5,0.1,1.5,1645.263158,1643.0,1155,1992,Converged : 19 times
159,8,0.5,0.1,1.5,2000.000000,2000.0,2000,2000,Did not converged : 81 times
160,8,0.5,0.2,0.5,9.750000,9.0,6,28,Converged : 100 times
192,8,0.5,0.2,1.0,395.000000,395.0,395,395,Converged : 1 times
161,8,0.5,0.2,1.0,2000.000000,2000.0,2000,2000,Did not converged : 99 times
162,8,0.5,0.2,1.5,2000.000000,2000.0,2000,2000,Did not converged : 100 times


### N1 = 10 Bipolar cross-entropy cost function  

In [16]:
df[df.N1==10]

,N1,weightspace,alpha,x0,mean_epoch,median_epoch,min_epoch,max_epoch,Result
224,10,0.5,0.1,0.5,162.666667,170.0,90,193,Converged : 24 times
195,10,0.5,0.1,0.5,2000.000000,2000.0,2000,2000,Did not converged : 76 times
225,10,0.5,0.1,1.0,660.193548,661.0,496,832,Converged : 31 times
196,10,0.5,0.1,1.0,2000.000000,2000.0,2000,2000,Did not converged : 69 times
226,10,0.5,0.1,1.5,1625.958333,1639.5,1288,1885,Converged : 24 times
197,10,0.5,0.1,1.5,2000.000000,2000.0,2000,2000,Did not converged : 76 times
198,10,0.5,0.2,0.5,8.520000,8.0,6,18,Converged : 100 times
199,10,0.5,0.2,1.0,2000.000000,2000.0,2000,2000,Did not converged : 100 times
200,10,0.5,0.2,1.5,2000.000000,2000.0,2000,2000,Did not converged : 100 times
201,10,0.5,0.3,0.5,6.707071,7.0,4,12,Converged : 99 times


## Observation


1. ***For - ζ and ζ in weightspace. It is observed that as ζ increase initial weight and bias range will result in fewer non-converge. It shows that a little larger weights and bias will encourage the better performance of our backpropagation model.***

2. ***For all test cases, As x0 increases, in most cases, the min, max, mean and median of epochs all increases which means that the model generally takes more iterations to converge.***

3. ***For all test cases, a larger learning rate will result in faster convergence and fewer iterations. At the same time, a larger learning rate will result in more non-convergence. This is reasonable because a large learning rate could substantially accelerate the learning process and if the step is too large(in other word, learning rate too large), the network will fail to converge.***
4. ***The larger the N1, the fewer epochs are required for convergence***

## --------------------------------------------------------------------

## N1=1 for Bipolar Cost Function

In [17]:
df[df.N1==1]

,N1,weightspace,alpha,x0,mean_epoch,median_epoch,min_epoch,max_epoch,Result
0,1,0.5,0.1,0.5,2000.0,2000.0,2000,2000,Did not converged : 100 times
1,1,0.5,0.1,1.0,2000.0,2000.0,2000,2000,Did not converged : 100 times
2,1,0.5,0.1,1.5,2000.0,2000.0,2000,2000,Did not converged : 100 times
3,1,0.5,0.2,0.5,2000.0,2000.0,2000,2000,Did not converged : 100 times
4,1,0.5,0.2,1.0,2000.0,2000.0,2000,2000,Did not converged : 100 times
5,1,0.5,0.2,1.5,2000.0,2000.0,2000,2000,Did not converged : 100 times
6,1,0.5,0.3,0.5,2000.0,2000.0,2000,2000,Did not converged : 100 times
7,1,0.5,0.3,1.0,2000.0,2000.0,2000,2000,Did not converged : 100 times
8,1,0.5,0.3,1.5,2000.0,2000.0,2000,2000,Did not converged : 100 times
9,1,1.0,0.1,0.5,2000.0,2000.0,2000,2000,Did not converged : 100 times


## Observations
***The model still cannot converge for the same reason***

### 2.2 For runs that do not converge (for either choice of the cost functions), what was the main problem?

## Solution:
There are three main reason for unconverge:
1. Because the XOR problem is not linearly separable. If there is only 1 neuron in hidden layer, the model cannot handle this problem.
2. Sometimes, for the reason that the initial weight are too small or the derivitive are too small, it takes many steps to converge and may exceed the max iteration time that we set.
3. Sometimes, for the reason of the larger learning rate, the model may have too wide step toward the target result to fail to converge in the end.